###Installation

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install python-dotenv
!pip install openai==0.28

In [4]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
os.environ["OPENAI_API_KEY"] = "API_KEY"

In [5]:
# llm_model = "gpt-3.5-turbo"

In [6]:
# !pip install langchain

###Creating Sequential Chain
Keywords are obtained from the Question and the topic

In [7]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

In [63]:
Topic = ["RESPIRATORY SYSTEM", "NEURAL CONTROL & COORDINATION", "RESPIRATORY SYSTEM"]
questions = ["The partial pressure of oxygen in the alveoli of the lung is","How do parasympathetic neural signals affect the working of the heart?","Name the chronic respiratory disorder caused mainly by cigarette smoking"]
Answers = ["more than that in the blood","Reduce both heart rate and cardiac output","Emphysema"]

In [9]:
from langchain import PromptTemplate
from langchain.chains import SequentialChain

In [57]:
llm = OpenAI(temperature=0)
template ='''
You are a very smart biology professor. \
Given a Topic and question, you can find all the pre-requisite needed to know in order to answer this question.
You are supposed to output a Paragraph on the pre-requisites.
Topic : {Topic}
Question :{Question}

'''
question_template = PromptTemplate(template = template, input_variables = ['Question','Topic'])
Question_chain = LLMChain(llm=llm, prompt=question_template,
                     output_key="Paragraph"
                    )

In [58]:
template ='''
You are a very smart biology professor. \
Given a paragraph, you can find all the keywords required to be known to understand this paragraph. \
provide an output of max 5 relevant keywords in the chrinological order required to understand this paragraph.

Paragraph :{Paragraph}
'''
keyword_template = PromptTemplate(template = template, input_variables = ['Paragraph'])
Q_Keywords_chain = LLMChain(llm=llm, prompt=keyword_template,
                     output_key="Keywords"
                    )

In [38]:
overall_chain = SequentialChain(
    chains = [Question_chain, Q_Keywords_chain],
    input_variables =["Question","Topic"],
    output_variables = ["Keywords"],
    verbose = True
)

In [39]:
result = overall_chain.run({"Question":questions[0],"Topic":Topic[0]})
result



> Entering new SequentialChain chain...

> Finished chain.


'\n1. Respiratory system\n2. Gas laws\n3. Diffusion\n4. Anatomy and physiology\n5. Homeostasis'

In [40]:
result = result.split("\n")
result.pop(0)
result

['1. Respiratory system',
 '2. Gas laws',
 '3. Diffusion',
 '4. Anatomy and physiology',
 '5. Homeostasis']

In [41]:
result

['1. Respiratory system',
 '2. Gas laws',
 '3. Diffusion',
 '4. Anatomy and physiology',
 '5. Homeostasis']

In [42]:
# pip install openai==0.28

####RAG implemented using Google Serper and Wikipedia
A detailed description of each keyword is obtained as learning material using agents


In [43]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [44]:
os.environ["SERPER_API_KEY"] = "55dde2f9ef1223219d597e7a579eb6cd82d79ce4"

In [45]:
tools = load_tools(["google-serper","wikipedia"], llm=llm)

In [46]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [48]:
response = ""
for i in range(5):
  ans = agent.run(f"""Provide a detailed description in 500 words on : {result[i]}
            """)
  response = response + ans + "\n"



> Entering new AgentExecutor chain...
Thought: I should use the wikipedia tool to get information on the respiratory system.
Action:
```
{
  "action": "wikipedia",
  "action_input": "Respiratory system"
}
```

Observation: Page: Respiratory system
Summary: The respiratory system (also respiratory apparatus, ventilatory system) is a biological system consisting of specific organs and structures used for gas exchange in animals and plants. The anatomy and physiology that make this happen varies greatly, depending on the size of the organism, the environment in which it lives and its evolutionary history. In land animals, the respiratory surface is internalized as linings of the lungs. Gas exchange in the lungs occurs in millions of small air sacs; in mammals and reptiles, these are called alveoli, and in birds, they are known as atria. These microscopic air sacs have a very rich blood supply, thus bringing the air into close contact with the blood. These air sacs communicate with the e

In [49]:
print(response)

The respiratory system is a biological system consisting of specific organs and structures used for gas exchange in animals and plants. It varies greatly depending on the organism, environment, and evolutionary history. In land animals, the respiratory surface is internalized as linings of the lungs. In most fish and other aquatic animals, the respiratory system consists of gills. Insects have a respiratory system with external openings called spiracles that lead to a network of tubes called tracheae. Reptiles are a group of tetrapods with an ectothermic metabolism and amniotic development, and their respiratory system is separate from the circulatory system.
The three gas laws in thermodynamics are Boyle's law, Charles's law, and Gay-Lussac's law. These laws describe the behavior of gases under different conditions, such as changes in temperature, pressure, and volume. They are fundamental principles in chemistry and physics, and are used to understand and predict the behavior of gase

In [50]:
import openai
import os


In [51]:
from langchain.llms.openai import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo")

The Detailed description of the keywords along with the solution to the question is concatenated

In [52]:
user_msg='Answer very precisely and in detailed format using tables,calculations if needed for the question(1000-2000 words) : ' + questions[0]
response_1 = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
              messages=[{"role": "system", "content": 'You are a helpful assistant which has some information from google in prompt, answer using this information and your own brain '},
                        {"role": "user", "content": user_msg}
              ])

In [53]:
print(response_1["choices"][0]["message"]["content"])

The partial pressure of oxygen in the alveoli of the lung is an important physiological parameter that is vital for efficient gas exchange in the respiratory system. In this response, we will explore the factors that influence the partial pressure of oxygen in the alveoli and how it is maintained within a specific range to support oxygen uptake and elimination of carbon dioxide.

To understand the partial pressure of oxygen in the alveoli, we must first understand the process of gas exchange that takes place during respiration. During inhalation, air containing oxygen enters the respiratory system through the nose or mouth, passes through the trachea, and reaches the alveoli in the lungs. The alveoli are small, thin-walled sacs surrounded by capillaries, where gas exchange between the lungs and the bloodstream occurs.

The oxygen in the inspired air diffuses across the alveolar-capillary membrane into the bloodstream, while carbon dioxide, a waste product of cellular metabolism, diffus

In [56]:
print(response)
print(response_1["choices"][0]["message"]["content"])

The respiratory system is a biological system consisting of specific organs and structures used for gas exchange in animals and plants. It varies greatly depending on the organism, environment, and evolutionary history. In land animals, the respiratory surface is internalized as linings of the lungs. In most fish and other aquatic animals, the respiratory system consists of gills. Insects have a respiratory system with external openings called spiracles that lead to a network of tubes called tracheae. Reptiles are a group of tetrapods with an ectothermic metabolism and amniotic development, and their respiratory system is separate from the circulatory system.
The three gas laws in thermodynamics are Boyle's law, Charles's law, and Gay-Lussac's law. These laws describe the behavior of gases under different conditions, such as changes in temperature, pressure, and volume. They are fundamental principles in chemistry and physics, and are used to understand and predict the behavior of gase

###Question 2

In [73]:
result = overall_chain.run({"Question":questions[1],"Topic":Topic[1]})
result



> Entering new SequentialChain chain...

> Finished chain.


'\n1. Parasympathetic neural signals\n2. Neural control and coordination\n3. Autonomic nervous system\n4. Sympathetic nervous system\n5. Parasympathetic nervous system'

In [74]:
result = result.split("\n")
result.pop(0)
result

['1. Parasympathetic neural signals',
 '2. Neural control and coordination',
 '3. Autonomic nervous system',
 '4. Sympathetic nervous system',
 '5. Parasympathetic nervous system']

In [75]:
response = ""
for i in range(5):
  ans = agent.run(f"""Provide a detailed description in 200 words on : {result[i]}
            """)
  response = response + ans + "\n"



> Entering new AgentExecutor chain...
Could not parse LLM output: Thought: I should use the wikipedia tool to get information on parasympathetic neural signals.
Action:
{
  "action": "wikipedia",
  "action_input": "parasympathetic neural signals"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should use the google_serper tool to get information on parasympathetic neural signals.
Action:
{
  "action": "google_serper",
  "action_input": "parasympathetic neural signals"
}


Observation: Invalid or incomplete response
Thought:Could not parse LLM output:  I should use the wikipedia tool to get information on parasympathetic nervous system.
Action:
{
  "action": "wikipedia",
  "action_input": "parasympathetic nervous system"
}



Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Could not parse LLM output:  I should use the wikipedia tool to get information on parasympathetic nervous system.
Action:
{
  "action": "wikipe

In [76]:
print(response)

Agent stopped due to iteration limit or time limit.
Neural control and coordination refers to the process by which the nervous system regulates and coordinates the activities of different organs and systems in the body. This includes both voluntary and involuntary actions, such as movement, digestion, and respiration. The nervous system achieves this through the use of specialized cells called neurons, which transmit signals throughout the body. These signals can be electrical or chemical in nature, and they allow for communication between different parts of the body. In animals, the nervous system is responsible for maintaining homeostasis and responding to external stimuli, allowing for survival and adaptation in changing environments. Disorders or damage to the nervous system can result in a variety of neurological conditions, affecting an individual's ability to control and coordinate their movements and bodily functions.
The autonomic nervous system (ANS) is a part of the peripher

In [77]:
user_msg='Answer very precisely and in detailed format using tables,calculations if needed for the question(1000-2000 words) : ' + questions[1]
response_1 = openai.ChatCompletion.create(
              model="gpt-3.5-turbo-16k",
              messages=[{"role": "system", "content": 'You are a helpful assistant which has some information from google in prompt, answer using this information and your own brain '},
                        {"role": "user", "content": user_msg}
              ])

In [78]:
print(response_1["choices"][0]["message"]["content"])

The parasympathetic neural signals affect the working of the heart through the autonomic nervous system, particularly the vagus nerve. The parasympathetic nervous system regulates various bodily functions, including heart rate, by releasing acetylcholine onto specific receptors in the heart. In this response, I will provide a detailed explanation of how parasympathetic neural signals influence the functioning of the heart.

To begin with, let's understand the autonomic nervous system, which consists of two branches: the sympathetic and parasympathetic nervous systems. These branches have antagonistic effects on various organs, including the heart. The sympathetic nervous system activates a "fight-or-flight" response, leading to increased heart rate and cardiac output, whereas the parasympathetic nervous system activates a "rest-and-digest" response, primarily slowing down heart activity.

The primary connection between the parasympathetic nervous system and the heart is mediated by the

In [79]:
print(response)
print(response_1["choices"][0]["message"]["content"])

Agent stopped due to iteration limit or time limit.
Neural control and coordination refers to the process by which the nervous system regulates and coordinates the activities of different organs and systems in the body. This includes both voluntary and involuntary actions, such as movement, digestion, and respiration. The nervous system achieves this through the use of specialized cells called neurons, which transmit signals throughout the body. These signals can be electrical or chemical in nature, and they allow for communication between different parts of the body. In animals, the nervous system is responsible for maintaining homeostasis and responding to external stimuli, allowing for survival and adaptation in changing environments. Disorders or damage to the nervous system can result in a variety of neurological conditions, affecting an individual's ability to control and coordinate their movements and bodily functions.
The autonomic nervous system (ANS) is a part of the peripher